In [10]:
import cv2
import mediapipe as mp
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
from collections import Counter
from sklearn.preprocessing import StandardScaler

In [11]:
# Inicializando o MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Pasta com os vídeos
pasta_videos = './completo - Copia'

# Listas para armazenar os dados e os rótulos
dados_direita = []
dados_esquerda = []
rotulos = []

In [12]:
# Dicionário para armazenar as coordenadas por palavra
coordenadas_por_palavra = {'direita': {}, 'esquerda': {}}

for subpasta in os.listdir(pasta_videos):
    for video in os.listdir(os.path.join(pasta_videos, subpasta)):
        # Lendo o vídeo
        cap = cv2.VideoCapture(os.path.join(pasta_videos, subpasta, video))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Convertendo a cor da imagem
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Processando a imagem
            result = hands.process(image)
            
            if result.multi_hand_landmarks and result.multi_handedness:
                coords_direita = None
                coords_esquerda = None
                
                for hand_landmarks, handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                    # Identificar se a mão é direita ou esquerda
                    hand_label = handedness.classification[0].label.lower()  # 'left' ou 'right'
                    if hand_label == 'right':
                        hand_label = 'direita'
                        coords_direita = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
                    else:
                        hand_label = 'esquerda'
                        coords_esquerda = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten())
                    
                # Adicionando as coordenadas à lista de dados
                if coords_direita:
                    dados_direita.append(coords_direita)
                if coords_esquerda:
                    dados_esquerda.append(coords_esquerda)
                    
                # Adicionando o rótulo à lista de rótulos
                rotulos.append(subpasta)
                
                # Armazenando as coordenadas por palavra e mão
                if subpasta not in coordenadas_por_palavra['direita']:
                    coordenadas_por_palavra['direita'][subpasta] = []
                if subpasta not in coordenadas_por_palavra['esquerda']:
                    coordenadas_por_palavra['esquerda'][subpasta] = []
                if coords_direita:
                    coordenadas_por_palavra['direita'][subpasta].append(tuple(coords_direita))
                if coords_esquerda:
                    coordenadas_por_palavra['esquerda'][subpasta].append(tuple(coords_esquerda))

# Selecionando os 30% movimentos mais frequentes por palavra e por mão
coordenadas_selecionadas = {'direita': {}, 'esquerda': {}}
for hand_label in ['direita', 'esquerda']:
    for palavra, coords_lista in coordenadas_por_palavra[hand_label].items():
        # Contar a frequência de cada conjunto de coordenadas
        contador = Counter(coords_lista)
        # Ordenar os conjuntos de coordenadas por frequência
        mais_frequentes = sorted(contador.items(), key=lambda item: item[1], reverse=True)
        # Selecionar os 30% conjuntos de coordenadas mais frequentes
        top_30_percent = mais_frequentes[:max(1, len(mais_frequentes) * 30 // 100)]
        coordenadas_selecionadas[hand_label][palavra] = [item[0] for item in top_30_percent]

# Preparando os dados para treinamento
dados_filtrados_direita = []
dados_filtrados_esquerda = []
rotulos_filtrados = []
for hand_label in ['direita', 'esquerda']:
    for palavra, coords_lista in coordenadas_selecionadas[hand_label].items():
        for coords in coords_lista:
            if hand_label == 'direita':
                dados_filtrados_direita.append(coords)
            else:
                dados_filtrados_esquerda.append(coords)
            rotulos_filtrados.append(palavra)

# Convertendo as listas em arrays numpy
dados_direita = np.array(dados_filtrados_direita)
dados_esquerda = np.array(dados_filtrados_esquerda)
rotulos = np.array(rotulos_filtrados)

# Normalizando os dados
scaler_direita = StandardScaler()
scaler_esquerda = StandardScaler()
dados_direita = scaler_direita.fit_transform(dados_direita)
dados_esquerda = scaler_esquerda.fit_transform(dados_esquerda)

# Balanceamento dos dados
dados_balanceados_direita = []
dados_balanceados_esquerda = []
rotulos_balanceados = []

for palavra in set(rotulos):
    indices_direita = [i for i, r in enumerate(rotulos) if r == palavra and i < len(dados_direita)]
    indices_esquerda = [i for i, r in enumerate(rotulos) if r == palavra and i < len(dados_esquerda)]
    min_amostras = min(len(indices_direita), len(indices_esquerda))
    if min_amostras > 0:
        indices_selecionados_direita = np.random.choice(indices_direita, min_amostras, replace=False)
        indices_selecionados_esquerda = np.random.choice(indices_esquerda, min_amostras, replace=False)
        dados_balanceados_direita.extend(dados_direita[indices_selecionados_direita])
        dados_balanceados_esquerda.extend(dados_esquerda[indices_selecionados_esquerda])
        rotulos_balanceados.extend(rotulos[indices_selecionados_direita])

dados_balanceados_direita = np.array(dados_balanceados_direita)
dados_balanceados_esquerda = np.array(dados_balanceados_esquerda)
rotulos_balanceados = np.array(rotulos_balanceados)

# Concatenando as coordenadas das mãos direita e esquerda
dados_balanceados = np.concatenate((dados_balanceados_direita, dados_balanceados_esquerda), axis=1)

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(dados_balanceados, rotulos_balanceados, test_size=0.2, random_state=42)

# Treinando o modelo com as coordenadas selecionadas
modelo = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=42, class_weight='balanced')
modelo.fit(X_train, y_train)

# Avaliando o modelo com validação cruzada
scores = cross_val_score(modelo, dados_balanceados, rotulos_balanceados, cv=5)
print(f'Acurácia média do modelo com validação cruzada: {scores.mean()*100:.2f}%')

# Avaliando o modelo
score = modelo.score(X_test, y_test)
print(f'Acurácia do modelo no conjunto de teste: {score*100:.2f}%')

# Salvando o modelo e os scalers
joblib.dump(modelo, 'modelo.pkl')
joblib.dump(scaler_direita, 'scaler_direita.pkl')
joblib.dump(scaler_esquerda, 'scaler_esquerda.pkl')

c:\Users\danil\Documents\GitHub\TCC2_modelo_IA\myenv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\danil\Documents\GitHub\TCC2_modelo_IA\myenv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Acurácia média do modelo com validação cruzada: 95.86%
Acurácia do modelo no conjunto de teste: 95.65%


['scaler_esquerda.pkl']